### 无序数组中求第K小的数

#### 方法一：改写快排

In [6]:
'''
思路：
1.随机选一个数V，做荷兰国旗问题。 时间O(N)，空间O(1)
2.看这个数V的等于区间是否包含K，若包含即为V。若不是根据K在哪侧，重新对哪侧选随机数，直到新区域包含K。
3.快排时间复杂度为O(NlogN)，因为跑了两侧。这里只需要跑一次，略证明，时间复杂度是O(N)
'''
import random
def minKth(array, k):
    arr = array.copy()
    return process(arr, 0, len(arr)-1, k-1) #k从1开始

def process(arr, L, R, index):
    if L == R: #==index
        return arr[L]
    pivot = arr[L+random.randint(0, R-L)]
    
    equalL, equalR = partition(arr, L, R, pivot)
    if index >= equalL and equalR >= index:
        return arr[index]
    elif index < equalL:
        return process(arr, L, equalL-1, index) #可以改迭代
    else:
        return process(arr, equalR+1, R, index)
    
def partition(arr, L, R, num):
    pass

#### 方法二：bfprt算法

In [ ]:
'''
改快排第一个选数阶段，替代随机选数：
bfprt(N, K)
1. 5个数一组，arr = [{abcde},{fghij},{klmno},{pqrs}] (19)
2. 每一组内部排序，5个数排序O(1)，有N/5个组， 所以代价O(N): arr=[{edbac},{jihfg},{mnolk},{rqps}]
3. 每个小组的中位数拿出来，组成新数组M，长度N/5. M = [b,h,o,q] (4)
4. 求M的中位数，即X = bfprt(M, N/10)  X = bfprt(M, 2)
5. X作为划分值进入快排中的partition

这样选择的X左右两侧：
1. 估计<X有几个数：求>=X有多少数，3N/10. 所以<X有7N/10个数
2. 同理>X有几个数：求<=X有多少数，3N/10. 所以>X有7N/10个数
3. 所以每次至少淘汰3N/10的数

整体复杂度
T(N) = T(N/5) + T(7N/10) + O(N) = O(N)
1. 5数一组O(N)
2. 组内排序O(N)
3. 中位数取出O(N/5)作为M数组
4. 求M数组中位数T(N*1/5)
5. partition过程O(N)
6. 如果未命中，重复1-5过程T(N*7/10)


'''
#arr[L..R] 如果排序的话，位于index位置的数是什么，返回
def bfprt(arr, L, R, index):
    if L == R:
        return arr[L]
    # L..R每五个数一组，每一个小组内部排好序，每个小组的中位数组成新数组，返回新数组的中位数
    pivot = medianOfMedians(arr, L, R)
    equalL, equalR = partition(arr, L, R, pivot)
    if index >= equalL and equalR >= index:
        return arr[index]
    elif index < equalL:
        return bfprt(arr, L, equalL-1, index) #可以改迭代
    else:
        return bfprt(arr, equalR+1, R, index) 
    
def medianOfMedians(arr, L, R):
    size = R - L + 1
    offset = 0 if size % 5 == 0 else 1
    mArr = [None] * (size // 5 + offset)
    for team in range(len(mArr)):
        teamFirst = L + team * 5
        mArr[team] = getMedian(arr, teamFirst, min(R, teamFirst+4))
        
    return bfprt(mArr, 0, len(mArr)-1, len(mArr) // 2)

def getMedian(arr, L, R):
    insertionSort(arr, L, R)
    return arr[(L+R)//2]
    
    

### 无序数组topK个最大的数
不同时间复杂度三个方法：
1. O(N*logN)：直接排序
2. O(N+K*logN)：从底向上将无序数组调整为大根堆代价O(N)+弹k次，每次大根堆调整O(logN)。整体O(N+K(logN)
3. O(N+K*logK)：先用bfprt找到第K大的数是多少，然后遍历数组，只要比这个数大的就记录，一定会找到<=K-1个数。不足K个的用第k大补。然后k个数内做排序返回

### 蓄水池算法
解决问题：假设有一个不断吐出不同球的机器，只有装下10个球的袋子，每次吐出的球，要么放入袋子，要么永远扔掉
如何做到机器吐出每个球之后，所有吐出的球都等概率被放进袋子里

In [ ]:
'''
1~10直接进袋子
1. 吐出i号球，以 10/i的概率决定要不要进袋子。f(i)：随机1~i，如果>10不进袋子。1~10进袋子。
2. 满袋子中10个球等概率弹出一个，i号球进袋子。

证明：
1729号球吐出，3号球还在袋子里的概率：
1~10，3进袋子的概率是1. 
11号球吐，3号弹出的概率是10/11(11进) * 1/10(3出) = 1/11，所以11号吐时，3在袋子的概率是10/11
12号球吐，3号弹出的概率是10/12(12进) * 1/10(3出) = 1/12，所以11号吐时，3在袋子的概率是11/12
...
1728号球吐，3号弹出的概率是10/1728(1728进) * 1/10(3出) = 1/1728，所以1728号吐时，3在袋子的概率是1727/1728
1729号球吐，3号弹出的概率是10/1729(1729进) * 1/10(3出) = 1/1729，所以1729号吐时，3在袋子的概率是1728/1729
累乘得到....1729号球吐时，3号球在袋子里的概率是 10/1729；

1729号球吐出，17号球还在袋子里的概率：
17号球要先进袋子，概率是10/17
18号球吐，17号弹出的概率是10/18(18进) * 1/10(17出) = 1/18，所以18号吐时，17在袋子的概率是17/18
...
1728号球吐，3号弹出的概率是10/1728(1728进) * 1/10(3出) = 1/1728，所以1728号吐时，3在袋子的概率是1727/1728
1729号球吐，3号弹出的概率是10/1729(1729进) * 1/10(3出) = 1/1729，所以1729号吐时，3在袋子的概率是1728/1729
累乘得到....1729号球吐时，3号球在袋子里的概率是 10/1729；

应用：
抽奖，1月1日0:00~1月2日0:00开奖，期间登陆过的用户等概率中奖。
'''

In [ ]:
'''
uuid系统吐出每一个号码一定不一样。同一时间可能有多个服务器，请求多个uuid。要求重复率0.
'''